In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import os
import cv2

In [ ]:
input_ex = torch.randn((1,3,244,244))
input_ex.shape

torch.Size([1, 3, 244, 244])

### 데이터 가져오기

In [ ]:
from pandas.core.frame import DataFrame
paths = []
dataset_type = []
labels = []

for dirname, _, filenames in os.walk('/content/drive/MyDrive/고모부_머신러닝/dogncat'):
    for filename in filenames:
        filepath = dirname + '/' + filename
        paths.append(filepath)

        if '/training_set' in filepath:
            dataset_type.append('train')

        elif '/test_set' in filepath:
            dataset_type.append('test')

        if 'dogs' in filepath:
            labels.append('DOG')
        
        elif 'cats' in filepath:
            labels.append('CAT')

In [ ]:
# Cat n Dog 데이터를 데이터 프레임으로 만들기(path, datatype, label)
cnd_df = pd.DataFrame({'path' : paths, 'type' : dataset_type, 'label' : labels})

# '.jpg'를 포함하는 path만 df로 만들기
filter = cnd_df['path'].str.contains('.jpg') 
cnd_df = cnd_df[filter]

# train, testset으로 분리
train_df = cnd_df[cnd_df['type'] == 'train']
test_df = cnd_df[cnd_df['type'] == 'test']

train_path = train_df['path'].values
test_path = test_df['path'].values

train_label = train_df['label'].values
test_label = test_df['label'].values


In [ ]:
train_label

array(['CAT', 'CAT', 'CAT', ..., 'DOG', 'DOG', 'DOG'], dtype=object)

In [ ]:
onehot_train_label = torch.tensor(pd.get_dummies(train_df['label']).values, dtype=torch.float32)
onehot_test_label = torch.tensor(pd.get_dummies(test_df['label']).values, dtype=torch.float32)


In [ ]:
train_label_indices = train_df['label'].replace(['CAT', 'DOG'], [0,1]).values
print(train_label_indices)
test_label_indices = test_df['label'].replace(['CAT', 'DOG'], [0,1]).values
print(test_label_indices)

[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]


### Custom Dataset만들기

In [ ]:
class MyDataset(Dataset):
    def __init__(self, path, label):
        super(MyDataset, self).__init__()
        self.paths = path
        self.labels = label

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        image = cv2.cvtColor(cv2.imread(self.paths[idx]), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (244,244))
        image = np.asarray(image, dtype=np.float32).transpose(2,0,1)

        if self.labels is not None:
            label = self.labels[idx]

        return image, label



### 모델 만들기

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.hidden1 = nn.Conv2d(3, 5, kernel_size=3, padding='same')
        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(3*244*244 ,2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        print('input shape:', x.shape)
        print(' input max : ', torch.amax(x, dim=(1,2,3)))
        print(' input min : ', torch.amin(x, dim=(1,2,3)))
        print(' input l2norm : ', torch.linalg.vector_norm(x, dim=(1,2,3)))

        # hidden layer
        conv = self.hidden1(x)
        print('conv result shape :', conv.shape)
        print(' conv max : ', torch.amax(conv, dim=(1,2,3)))
        print(' conv min : ', torch.amin(conv, dim=(1,2,3)))
        print(' conv l2norm : ', torch.linalg.vector_norm(conv, dim=(1,2,3)))
        conv_out = self.relu(conv)
        print('conv_out shape :', conv_out.shape)
        print(' conv_out max : ', torch.amax(conv_out, dim=(1,2,3)))
        print(' conv_out min : ', torch.amin(conv_out, dim=(1,2,3)))
        print(' conv_out l2norm : ', torch.linalg.vector_norm(conv_out, dim=(1,2,3)))

        fc_input = x.view(x.size(0), -1)
        print('fc_input shape :', fc_input.shape)
        print(' fc_input max : ', torch.amax(fc_input, dim=1))
        print(' fc_input min : ', torch.amin(fc_input, dim=1))
        print(' fc_input l2norm : ', torch.linalg.vector_norm(fc_input, dim=1))

        # output layer
        fc_logit = self.fc1(fc_input)
        print('fc_logit shape :', fc_logit.shape)
        print(' fc_logit max : ', torch.amax(fc_logit, dim=1))
        print(' fc_logit min : ', torch.amin(fc_logit, dim=1))
        print(' fc_logit l2norm : ', torch.linalg.vector_norm(fc_logit, dim=1))
        fc_output = self.softmax(fc_logit)
        print('fc_output :', fc_output)
        print('fc_output shape :', fc_output.shape)
        print(' fc_output max : ', torch.amax(fc_output, dim=1))
        print(' fc_output min : ', torch.amin(fc_output, dim=1))
        print(' fc_output l2norm : ', torch.linalg.vector_norm(fc_output, dim=1))

        return fc_logit
        

In [ ]:
model = CNN()
model

CNN(
  (hidden1): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (relu): ReLU()
  (fc1): Linear(in_features=178608, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

In [ ]:
cnd_dataset = MyDataset(train_path, train_label_indices)
loader = DataLoader(cnd_dataset, batch_size=10, shuffle=True)
image_data, label_data = next(iter(loader))
print(image_data.view(image_data.size(0), -1))
print(label_data)

tensor([[194., 191., 189.,  ...,  31.,  32.,  33.],
        [ 84.,  83.,  83.,  ..., 168., 168., 168.],
        [145., 151., 126.,  ..., 115., 114., 115.],
        ...,
        [ 81.,  76.,  76.,  ...,  44.,  50.,  37.],
        [226., 252., 243.,  ...,  75.,  76.,  53.],
        [212., 212., 213.,  ...,  96.,  92.,  94.]])
tensor([1, 0, 1, 0, 1, 0, 0, 1, 1, 0])


### CrossEntropyLoss 사용
- input 데이터 : raw, unnormalized data. output layer에서 activation function으로 softmax를 적용하기 전 logit을 input으로 전달한다.
- target 데이터 : indices class. 각각의 class에 대해 index번호가 매겨진 데이터
- 참조  
https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html?highlight=crossentropy#torch.nn.CrossEntropyLoss

In [ ]:
loss_fn = nn.CrossEntropyLoss(reduction='mean')
for _ in range(5):
    image_data, label_data = next(iter(loader))
    hyphothesis = model(image_data)
    loss = loss_fn(hyphothesis, label_data)
    # print('hyphothesis :\n' ,hyphothesis)
    print('loss : \n', loss)

input shape: torch.Size([10, 3, 244, 244])
 input max :  tensor([168., 255., 255., 255., 255., 220., 255., 255., 239., 238.])
 input min :  tensor([ 0.,  0.,  0.,  2., 78.,  0.,  0.,  0.,  0.,  0.])
 input l2norm :  tensor([28804.8516, 59670.8125, 59410.4062, 49933.3984, 83045.0156, 53540.6875,
        36942.7500, 64919.6523, 64001.0039, 29392.0430])
conv result shape : torch.Size([10, 5, 244, 244])
 conv max :  tensor([230.0371, 418.4395, 330.2481, 314.6115, 385.9077, 284.3976, 403.2035,
        332.3542, 329.5766, 284.0697], grad_fn=<AmaxBackward0>)
 conv min :  tensor([ -96.5343, -137.6742, -167.1531,  -77.6474, -159.6629, -103.3288,
        -134.5518, -139.8193, -134.0641, -119.0906], grad_fn=<AminBackward0>)
 conv l2norm :  tensor([22175.7344, 47443.0273, 45555.9648, 39738.1562, 64834.5117, 41881.2969,
        28453.8242, 46941.9727, 51059.8164, 23192.2246],
       grad_fn=<LinalgVectorNormBackward0>)
conv_out shape : torch.Size([10, 5, 244, 244])
 conv_out max :  tensor([230.0371

In [ ]:
torch.exp(torch.tensor(26.6739))

tensor(3.8400e+11)

### CrossEntropyLoss 예제(pytorch document)

In [ ]:
loss_p = nn.CrossEntropyLoss(reduction='mean')
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss_p(input, target)

In [ ]:
print(input)
print()
print(target)
print(output)

tensor([[-0.0585, -0.1539,  0.2935, -0.6673,  0.9894],
        [ 2.1771, -0.1619, -0.2550,  0.0030, -1.6859],
        [ 0.5653, -0.9964, -0.5865, -0.8744, -1.6881]], requires_grad=True)

tensor([3, 0, 2])
tensor(1.5228, grad_fn=<NllLossBackward0>)


In [ ]:
target2 = torch.randn(3,5).softmax(dim=1)
output2 = loss(input, target2)

print(target2)
print(output2)

tensor([[0.2395, 0.1087, 0.0064, 0.1246, 0.5209],
        [0.0881, 0.1126, 0.5270, 0.0960, 0.1763],
        [0.1382, 0.0414, 0.2380, 0.4794, 0.1029]])
tensor(1.6698, grad_fn=<DivBackward1>)


normalized -> 0~1사이의 값으로 만들어주는 것

scheduler method = linear.  
optimizer = SGD(lr : 0.001).  
1step training


### softmax
공식
$$Softmax(x_i) = {\exp(x_i) \over\sum_{j=1}^{n} \exp(x_j)}$$
n = class 개수

### 엔트로피
- ex) 따뜻한 물과 차가운 물을 섞을 경우
    - 섞기 전 : 엔트로피 min
    - 완전히 섞은 후 : 엔트로피 max

- Uniform probalility Distribution
    - 치우치지 않고 완벽하게 섞인 상태. 즉, 어떤 사건이 발생할 확률이 동일한 상태
    - Maximum entropy

- Expectation 공식
$$E(x) = \sum_{i=1}^{n} x_i p(x_i)  $$
    - ex) X = {x1, x2}, x1 = 1, x2 = 0  
\begin{align}
E(X) 
& = \sum_{i=1}^{2} x_i p(x_i)\\
& = x_1 p(x_1) + x_2 p(x_2)\\
& = x_1 p(x_1) + (1 - x_2)) p(x_2)\\
\end{align}

- Entorpy 공식
$$E(X) = \sum_{i=1}^{n} p(x_i) log_2(1/p(x_i))$$
    - 데이터를 비트로 표현한 것 :  $$log_2(1/p(x_i))$$
    - 즉, 엔트로피는 데이터를 표현하는 비트 개수의 평균이라고 할 수 있다?